In [260]:
from selenium import webdriver
from bs4  import BeautifulSoup
import time
import csv
import os
import shutil
import codecs
import pandas as pd
import numpy as np

In [261]:
data = r'C:\Users\preya\Downloads\Telegram Desktop\ChatExport_2021-06-27 (1)'
file_paths = []
#loop to get append file_paths 
for root, directories, files in os.walk(data):
    for filename in files:
        if ".html" in filename:
            # Join the two strings in order to form the full filepath.
            filepath = os.path.join(root, filename)
            #to append
            file_paths.append(filepath)

In [262]:
messages = []
for file in file_paths:    
    soup = BeautifulSoup(codecs.open(file, 'r', "utf-8").read())
    messages.extend(soup.findAll('div', attrs={'class':'message default clearfix'}))
    messages.extend(soup.findAll('div', attrs={'class':'message default clearfix joined'}))
    
print(len(contents))

4500


In [263]:
current_ids, previous_ids, message_texts,message_statuses = [],[],[],[]

for meg in messages:
    current_id = meg.get("id")[7:]
    current_id = int(current_id)
    
    try:
        previous_id = meg.find("div", {"class":"reply_to details"}).a.get("href")[14:]
        previous_id = int(previous_id)
    except:
        previous_id = 0
    
    try:
        message_text = meg.find("div", {"class":"description"})
        message_text = message_text.text.strip()
    except:
        try:
            message_text = meg.find("div", {"class":"text"})
            message_text = message_text.text.strip()
        except:
            continue
    
    try:
        message_status = meg.find("div", {"class":"status details"})
        message_status = message_status.text.strip().split(",")[0]
    except:
        message_status = None
    
    if "Not included, change data exporting settings to download." not in message_text and "/" not in message_text and "Try command /" not in message_text:
        current_ids.append(current_id)
        previous_ids.append(previous_id)
        message_texts.append(message_text)
        message_statuses.append(message_status)

In [264]:
dataset = pd.DataFrame({
    "current_id":current_ids,
    "previous_id":previous_ids,
    "message_text":message_texts,
    "message_status":message_statuses
    })

In [265]:
np.sum(dataset.isnull())

current_id             0
previous_id            0
message_text           0
message_status    413818
dtype: int64

In [266]:
def readPreviousMain(currentID):
    pm = dataset[dataset["previous_id"] == currentID].iloc[:,2].values
    pm = str(pm)

    cm = dataset[dataset["current_id"] == currentID].iloc[:,2].values
    cm = str(cm)

    print("Previous Message: ", pm)
    print("Current Message: ", cm)

In [267]:
readPreviousMain(5500021)

Previous Message:  []
Current Message:  ['I m also free You got free from ?']


In [268]:
readPreviousMain(5500036)

Previous Message:  []
Current Message:  ['Hi']


In [269]:
readPreviousMain(5500023)

Previous Message:  ['Chhii Omg']
Current Message:  ['Vo to funny hai sirf 😂🙈👀']


In [270]:
dataset.to_csv(r'C:\Users\preya\Downloads\Telegram Desktop\ChatExport_2021-06-27 (1)\dataset.csv')

,current_id,previous_id,message_text,message_status
0,5500016,5491000,😂,None
1,5500021,5500000,I m also free You got free from ?,None
2,5500023,5500006,Vo to funny hai sirf 😂🙈👀,None
3,5500031,5494639,शुक्रिया,None
4,5500036,5499996,Hi,None
...,...,...,...,...
413828,5688350,0,Wbu?,None
413829,5688385,0,Koi hamse bhi baate Kar lo,None
413830,5688388,0,😂😂,None
413831,5688527,0,??,None


In [271]:
dataset.to_csv(r'C:\Users\preya\Downloads\Telegram Desktop\ChatExport_2021-06-27 (1)\dataset.csv')

In [273]:
dataset.head()

,current_id,previous_id,message_text,message_status
0,5500016,5491000,😂,None
1,5500021,5500000,I m also free You got free from ?,None
2,5500023,5500006,Vo to funny hai sirf 😂🙈👀,None
3,5500031,5494639,शुक्रिया,None
4,5500036,5499996,Hi,None


DilogFlow

In [278]:
def create_intent(project_id, display_name, training_phrases_parts, message_texts):
    """Create an intent of the given intent type."""
    from google.cloud import dialogflow

    intents_client = dialogflow.IntentsClient()

    parent = dialogflow.AgentsClient.agent_path(project_id)
    training_phrases = []
    for training_phrases_part in training_phrases_parts:
        part = dialogflow.Intent.TrainingPhrase.Part(text=training_phrases_part)
        # Here we create a new training phrase for each provided part.
        training_phrase = dialogflow.Intent.TrainingPhrase(parts=[part])
        training_phrases.append(training_phrase)

    text = dialogflow.Intent.Message.Text(text=message_texts)
    message = dialogflow.Intent.Message(text=text)

    intent = dialogflow.Intent(
        display_name=display_name, training_phrases=training_phrases, messages=[message]
    )

    response = intents_client.create_intent(
        request={"parent": parent, "intent": intent}
    )

    print("Intent created: {}".format(response))

In [279]:
training_phrases_parts = ["what is your name?", "your name?", "whats your good name?"]
message_texts = "My name is preyash"
create_intent(
    project_id="infigontest",
    display_name="infigonTest",
    training_phrases_parts = training_phrases_parts,
    message_texts = message_texts
)

DefaultCredentialsError: Could not automatically determine credentials. Please set GOOGLE_APPLICATION_CREDENTIALS or explicitly create credentials and re-run the application. For more information, please see https://cloud.google.com/docs/authentication/getting-started